In [ ]:
import requests
import re

In [ ]:
with open(r"C:\Riot Games\League of Legends\lockfile", 'r') as infile:
    _,_,port,password,protocol = infile.read().split(':')

base_url = f'{protocol}://127.0.0.1:{port}'

s = requests.Session()
s.auth = ('riot', password)
s.verify = False

response = s.post(f'{base_url}/Help', params={'format': 'Console'})

In [ ]:
to_snake_case = re.compile(r'(?<!^)(?=[A-Z])')

translator = {
    'Bool': 'bool',
    'Int64': 'i64',
    'Int32': 'i32',
    'Int16': 'i16',
    'Int8': 'i8',
    'Uint64': 'u64',
    'Uint32': 'u32',
    'Uint16': 'u16',
    'Uint8': 'u8',
    'Float': 'f32',
    'Double': 'f64',
    'String': 'String',
    'Object': 'Value',
}

def format_var_name(var_name: str, snake_case=True) -> tuple[str, str | None]:
    if var_name in ['type', 'async', 'in']:
        alias = var_name
        var_name += '_'
    elif '-' in var_name:
        alias = var_name
        var_name = '_'.join(var_name.split('-'))
    else:
        alias = None

    if snake_case:
        var_name = to_snake_case.sub('_', var_name).lower()

    return (var_name, alias)

def format_type_name(type_name: str) -> str:
    split = re.split(r'[\-_ ]', type_name)
    return ''.join([x[0].upper() + x[1:] if len(x) > 1 else x.upper() for x in split])

def determine_var_type(var_type: str, optional: bool) -> str:
    if var_type.startswith('vector of '):
        var_type = var_type.replace('vector of ', '')
        coll = 'Vec'
    elif var_type.startswith('map of '):
        var_type = var_type.replace('map of ', '')
        coll = 'HashMap'
    else:
        coll = None

    var_type = format_type_name(var_type)

    if var_type in translator:
        var_type = translator[var_type]

    if coll == "HashMap":
        var_type = f"HashMap<String, {var_type}>"
    elif coll == "Vec":
        var_type = f"Vec<{var_type}>"

    if optional:
        var_type = f"Option<{var_type}>"

    return var_type

In [ ]:
json: dict[str, dict[str, list[dict]]] = response.json()['types']

json['LolEventShopTokenUpsellLockedType']['values'][2]['value'] = 2
json['LolWorldsTokenCardTokenUpsellLockedType']['values'][2]['value'] = 2

types = {}
enums = {}
for type_, definition in json.items():
    if 'fields' in definition:
        type_dict = {}
        for var in definition['fields']:
            var_name = list(var.keys())[0]
            var = var[var_name]
            var_type = var['type']
            var_name, alias = format_var_name(var_name)

            if type(var_type) == str:
                var_type = determine_var_type(var_type, var['optional'])
            elif type(var_type) == dict:
                var_type = determine_var_type(list(var_type.keys())[0], var['optional'])

            type_dict[var_name] = {
                'alias': alias,
                'type': var_type
            }

        types[format_type_name(type_)] = type_dict
    elif 'values' in definition:
        enums_list = []
        for val in definition['values']:
            enum_name, alias = format_var_name(val['name'], snake_case=False)
            enums_list.append({
                'enum_name': enum_name,
                'value': val['value'],
                'alias': alias
            })
        enums[format_type_name(type_)] = enums_list

In [ ]:
types_string = """#![allow(dead_code, non_snake_case, non_camel_case_types)]
use std::collections::HashMap;
use serde_json::Value;
use serde::{Deserialize, Serialize};

"""
for type_, definition in types.items():
    vars_string = ',\n    '.join([(f'#[serde(rename = "{definition[var]["alias"]}")]\n    ' if definition[var]["alias"] is not None else '') + f"pub {var}: {definition[var]['type']}" for var in definition])
    types_string += f"""#[derive(Deserialize, Serialize, Debug, Clone, Default)]
#[serde(rename_all = "camelCase")]
pub struct {type_} {{
    {vars_string}
}}

"""

for enum, definition in enums.items():
    enums_string = ',\n    '.join([f"{enum['enum_name']} = {enum['value']}" for enum in definition])
    types_string += f"""#[derive(Deserialize, Serialize, Debug, Clone)]
pub enum {enum} {{
    {enums_string}
}}

"""

In [ ]:
with open(f"../src/api/api_types.rs", 'w') as outfile:
    outfile.write(types_string)

In [ ]:
functions = {}
for func, description in response.json()['functions'].items():
    if 'url' in description:
        func_dict = {
            'url': description['url'],
            'path_params': [],
            'query_params': [],
            'body_param': None,
            'http_method': description['http_method'].lower()
        }
        if 'returns' in description:
            returns = description['returns']
            if type(returns) == str:
                returns = determine_var_type(returns, False)
            else:
                returns = determine_var_type(list(returns.keys())[0], False)

            func_dict['returns'] = returns
        else:
            func_dict['returns'] = None

        for arg in description['arguments']:
            arg_name = list(arg.keys())[0]

            arg_def = {
                'original_name': arg_name
            }

            arg_name_snake, _ = format_var_name(arg_name)
            arg_def['name'] = arg_name_snake

            body_arg = False

            arg_type = arg[arg_name]['type']

            if 'optional' in arg[arg_name]:
                optional = arg[arg_name]['optional']
            else:
                optional = False


            if type(arg_type) == str:
                name_in_collection = arg_type.replace('vector of ', '').replace('map of ', '')
                if name_in_collection.title() not in translator and name_in_collection not in enums:
                    body_arg = True
                arg_type = determine_var_type(arg_type, optional)
            else:
                arg_type = list(arg_type.keys())[0]
                arg_type = determine_var_type(arg_type, False)
                if format_type_name(arg_type) not in enums:
                    body_arg = True

            arg_def['type'] = arg_type if arg_type != "" else "String"

            if f"{{{arg_name}}}" in description['url']:
                func_dict['url'] = description['url'].replace(f"{{{arg_name}}}", f"{{{{{arg_name_snake}}}}}")
                func_dict['path_params'].append(arg_def)
            elif body_arg:
                func_dict['body_param'] = arg_def
            else:
                func_dict['query_params'].append(arg_def)
                
        func_name, _ = format_var_name(func)
        functions[func_name] = func_dict

for func, description in functions.items():
    if description['body_param'] is not None and description['http_method'] in ['GET', 'DELETE', 'POST'] and description['returns'] is not None:
        print(func, description)

In [ ]:
functions['get_lol_ranked_v2_tiers']

In [ ]:
out_string = """#![allow(dead_code)]
use std::error::Error;
use std::collections::HashMap;
use handlebars::Handlebars;
use serde_json::json;
use serde_json::value::Value;
use crate::rest::RESTClient;
use super::api_types::*; 

"""
for func, description in functions.items():
    returns = '()'
    if description['returns'] is not None:
        returns = description['returns']

    params = ["client: &RESTClient"]

    for path_param in description['path_params']:
        params.append(f"{path_param['name']}: {path_param['type']}")

    for query_param in description['query_params']:
        params.append(f"{query_param['name']}: {query_param['type']}")

    if description['body_param'] is not None:
        params.append(f"body: {description['body_param']['type']}")

    params = ', '.join(params)

    lines = []
    

    if description['body_param'] is not None:
        lines.append("let body = serde_json::to_value(&body)?;")

    url = description["url"]
    if len(description['path_params']) > 0 or len(description['query_params']) > 0:
        lines.append('let reg = Handlebars::new();')

        query_parts = '&'.join([f'{x["original_name"]}={{{{{x["name"]}}}}}' for x in description["query_params"]])
        if len(query_parts) > 0:
            url = f"{url}?{query_parts}"

        template_json_path = [f'"{x["name"]}": {x["name"]}' for x in description["path_params"]]
        template_json_query = [f'"{x["name"]}": {x["name"]}' for x in description["query_params"]]
        template_json = ", ".join(template_json_path + template_json_query)

        lines.append(f'let template_url = reg.render_template("{url}", &json!({{{template_json}}}))?;');
        lines.append(f'let url = format!("{{}}", template_url);')
    else:
        lines.append(f'let url = "{url}";')

    request_args = ['url.to_owned()']
    if description['body_param'] is not None:
        request_args.append('Some(body)')
    elif description['http_method'] in ['post', 'put', 'patch']:
        request_args.append('None')

    request_args = ', '.join(request_args)

    if description['returns'] is not None:
        lines.append(f'let value = client.{description["http_method"]}({request_args}).await?;')
    else:
        lines.append(f'client.{description["http_method"]}({request_args}).await?;')

    if description['returns'] is None:
        lines.append('Ok(())')
    elif returns == "Value":
        lines.append('Ok(value)')
    else:
        lines.append(f'Ok(serde_json::from_value::<{returns}>(value)?)')

    lines = '\n    '.join(lines)
    
    out_string += f"""pub async fn {func}({params}) -> Result<{returns}, Box<dyn Error>> {{
    {lines}
}}

"""
print(out_string)

In [ ]:
with open(f"../src/api/api_endpoints.rs", 'w') as outfile:
    outfile.write(out_string)